In [5]:
from datasets import SHD_dataloaders, SSC_dataloaders, GSC_dataloaders
from config import Config
from snn_delays import SnnDelays
from snn_delays_daleian import SnnDelays_Dale
from snn_daleian import SNN_Dale
import torch
from snn import SNN
import utils

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n=====> Device = {device} \n\n")


=====> Device = cpu 




In [16]:
config = Config()

if config.model_type == 'snn':
    model = SNN(config).to(device)
if config.model_type == 'snn_delays_dale':
    model = SnnDelays_Dale(config).to(device)
if config.model_type == 'snn_dale':
    model = SNN_Dale(config).to(device)
else:
    model = SnnDelays(config).to(device)

if config.model_type == 'snn_delays_lr0':
    model.round_pos()


print(f"===> Dataset    = {config.dataset}")
print(f"===> Model type = {config.model_type}")
print(f"===> Model size = {utils.count_parameters(model)}\n\n")

===> Dataset    = shd
===> Model type = snn_dale
===> Model size = 107520




In [9]:
if config.dataset == 'shd':
    train_loader, valid_loader = SHD_dataloaders(config)
    test_loader = None
elif config.dataset == 'ssc':
    train_loader, valid_loader, test_loader = SSC_dataloaders(config)
elif config.dataset == 'gsc':
    train_loader, valid_loader, test_loader = GSC_dataloaders(config)
else:
    raise Exception(f'dataset {config.dataset} not implemented')

The directory [Datasets/SHD/extract] for saving extracted files already exists.
SpikingJelly will not check the data integrity of extracted files.
If extracted files are not integrated, please delete [Datasets/SHD/extract] manually, then SpikingJelly will re-extract files from [Datasets/SHD/download].
The directory [Datasets/SHD/duration_10] already exists.
The directory [Datasets/SHD/extract] for saving extracted files already exists.
SpikingJelly will not check the data integrity of extracted files.
If extracted files are not integrated, please delete [Datasets/SHD/extract] manually, then SpikingJelly will re-extract files from [Datasets/SHD/download].
The directory [Datasets/SHD/duration_10] already exists.


In [17]:
#model.train_model(train_loader, valid_loader, test_loader, device)

In [10]:
# Get one batch from the train_loader
train_iter = iter(train_loader)
first_batch = next(train_iter)

In [11]:
# Assuming the batch is a tuple of (data, labels)
data, labels, _ = first_batch

# Print the first data point and its label
print(data[0], labels[0])

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64) tensor(0)


In [12]:
data.size()

torch.Size([256, 137, 140])

In [13]:
data = data.permute(1,0,2).float().to(device)  #(time, batch, neurons)

In [12]:
data.size()

torch.Size([121, 256, 140])

In [ ]:
output = model.forward(data)

In [ ]:
output

In [ ]:
x = torch.rand(10, 2) > 0.2
x

In [ ]:
x[:, 1]

In [ ]:
model.train_model(train_loader, valid_loader, test_loader, device)

# **Nouvelle classe linéaire**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
from DaleLinear import DaleLinear

In [4]:
torch.empty(3)

tensor([2.7446e-34, 0.0000e+00, 2.8191e-34])

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class DANNLayer(nn.Module):
    def __init__(self, in_features, out_features, exc_proportion, bias, step_mode):
        super().__init__()

        self.in_features = in_features
        
        self.in_features_exc = round(in_features * exc_proportion)
        self.in_features_inh = in_features - self.in_features_exc

        # Initialize parameters
        self.WEE = nn.Parameter(torch.empty(self.in_features_exc, self.in_features_exc))
        self.WIE = nn.Parameter(torch.empty(self.in_features_inh, self.in_features_exc))
        self.WEI = nn.Parameter(torch.empty(self.in_features_exc, self.in_features_inh))
        self.alpha = nn.Parameter(torch.empty(self.in_features_inh, 1))
        self.g = nn.Parameter(torch.empty(self.in_features_exc, 1))
        self.beta = nn.Parameter(torch.empty(self.in_features_exc, 1))

        self.step_mode = step_mode

        self.init_weights()

    def init_weights(self):
        lambda_E = math.sqrt(self.in_features * (2 * math.pi - 1) / (2 * math.pi))
        exponential_dist = torch.distributions.exponential.Exponential(lambda_E)

        with torch.no_grad():
            self.WEE.copy_(exponential_dist.sample(self.WEE.shape))
        
            if self.in_features_inh == 1:
                self.WIE.fill_((1 / self.in_features_exc) * torch.sum(self.WEE, dim=1))
                self.WEI.fill_(1)
            else:
                self.WIE.copy_(exponential_dist.sample(self.WIE.shape))
                self.WEI.fill_(1 / self.in_features_exc)
        
            self.alpha.fill_(math.log(math.sqrt(2 * math.pi - 1) / math.sqrt(self.in_features)))
            self.g.fill_(1)
            self.beta.fill_(1)

    def forward(self, x):
        # x is expected to be of shape (time, batch_size, ne)
        
        zE = torch.mm(self.WEE, x.T) #(ne, batch_size)
        hI = torch.mm(self.WIE, x.T) #(ni, batch_size)
        
        inhibition_effect = torch.mm(self.WEI, hI) #(ne, batch_size)
        
        gamma = torch.mm(self.WEI,  hI * torch.exp(self.alpha)) + 1e-8 #(ne, batch_size)

        z = (self.g/gamma) * (zE - inhibition_effect) + self.beta.T #(ne, batch_size)
        
        h = F.relu(z) #(ne, batch_size)
        
        return h

        

    def update_weights(self, eta, grads):
        with torch.no_grad():
            self.WEE -= eta * grads['WEE']
            self.WIE -= eta * grads['WIE']
            self.WEI -= eta * grads['WEI']
            self.alpha -= eta * grads['alpha']
            self.g -= eta * grads['g']
            self.beta -= eta * grads['beta']
            
            self.WEE.data = torch.clamp(self.WEE.data, min=0)
            self.WIE.data = torch.clamp(self.WIE.data, min=0)
            self.WEI.data = torch.clamp(self.WEI.data, min=0)
            self.g.data = torch.clamp(self.g.data, min=0)

    def correct_gradients(self, grads):
        grads['WEE'] = grads['WEE']
        grads['WIE'] = grads['WIE'] / math.sqrt(self.ne)
        grads['WEI'] = grads['WEI'] / self.in_features
        grads['alpha'] = grads['alpha'] / (self.in_features * math.sqrt(self.ne))
        grads['g'] = grads['g']
        grads['beta'] = grads['beta']
        return grads

In [18]:
x = torch.rand([100, 256, 50])

In [27]:
D = DANNLayer(50, 30, 0.8, bias=False, step_mode='m')

In [28]:
D(x)

RuntimeError: mat2 must be a matrix

In [4]:
#l = [['u', 'i']]
l = ['u']

for i in l[0]:

    print(type(i))

<class 'str'>
